In [ ]:
from __Detection.detection_api import DetectionAPI
from __Tracking.tracking_api import TrackerAPI
from homo_graphy import PlanProjector
import cv2

In [2]:
detector = DetectionAPI()

Loads checkpoint by local backend from path: /workspace/PretrainedModel_by_JeonYT/vehicle/epoch_54.pth
Loads checkpoint by local backend from path: /workspace/PretrainedModel_by_JeonYT/worker/epoch_100.pth


In [3]:
class Args:
    track_thresh = 0.5
    match_thresh = 0.5
    track_buffer = 60
    mot20 = False
args = Args()

In [7]:
tracker = TrackerAPI(args=args, detector=detector)
tracklets = tracker.trackingVideo(
    video_path="/workspace/mounted_datasets/Hyundai_dataset/aug1/c1/video.mp4",
    save_path="/workspace/results/tracking_result2.mp4",
    trail_len=100
)

결과 영상 저장 완료: /workspace/results/tracking_result2.mp4


In [ ]:
# 1) 도면 + (선택) H 로드
plan_path = "/workspace/plan.png"
projector = PlanProjector(plan_path, trail_len=60)


# 2) 대응점으로 H 추정 (리스트 안에 '튜플' 형태) --> 예시 나중에 바꿔야함
image_pts = [(100, 200), (400, 210), (395, 600), (120, 590)]
plan_pts  = [( 10,  20), ( 80,  20), ( 80, 120), ( 10, 120)]
H, mask = projector.fit_homography(image_pts, plan_pts, ransac_thresh=3.0)

# 4) 한 프레임 투영 & 저장
projected, canvas = projector.projection(tracklets, mode="center", draw=True)
cv2.imwrite("/workspace/plan_proj_frame0.jpg", canvas)

# 5) 전체 프레임 미니맵 비디오 저장
projector.save_video(tracklets, "/workspace/plan_projection.mp4", fps=30, mode="center")
